# API Gallery

The txtai API is a web-based service backed by [FastAPI](https://fastapi.tiangolo.com/). All txtai functionality including similarity search, extractive QA and zero-shot labeling is available via the API.

This notebook installs the txtai API and shows an example using each of the supported language bindings for txtai.

# Install dependencies

Install `txtai` and all dependencies. Since this notebook uses the API, we need to install the api extras package.

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[api]

# Python

The first method we'll try is direct access via Python. We'll use zero-shot labeling for all the examples here. See [this notebook](https://colab.research.google.com/github/neuml/txtai/blob/master/examples/07_Apply_labels_with_zero_shot_classification.ipynb) for more details on zero-shot classification. 

## Configure Labels instance

In [ ]:
%%capture
import os
from IPython.core.display import display, HTML
from txtai.pipeline import Labels

def table(rows):
    html = """
    <style type='text/css'>
    @import url('https://fonts.googleapis.com/css?family=Oswald&display=swap');
    table {
      border-collapse: collapse;
      width: 900px;
    }
    th, td {
        border: 1px solid #9e9e9e;
        padding: 10px;
        font: 20px Oswald;
    }
    </style>
    """

    html += "<table><thead><tr><th>Text</th><th>Label</th></tr></thead>"
    for text, label in rows:
        html += "<tr><td>%s</td><td>%s</td></tr>" % (text, label)
    html += "</table>"

    display(HTML(html))

# Create labels model
labels = Labels()

## Apply labels to text

In [ ]:
data = ["Wears a red suit and says ho ho",
        "Pulls a flying sleigh",
        "This is cut down and decorated",
        "Santa puts these under the tree",
        "Best way to spend the holidays"]

# List of labels
tags = ["🎅 Santa Clause", "🦌 Reindeer", "🍪 Cookies", "🎄 Christmas Tree", "🎁 Gifts", "👪 Family"]

# Render output to table
table([(text, tags[labels(text, tags)[0][0]]) for text in data])

Text,Label
Wears a red suit and says ho ho,🎅 Santa Clause
Pulls a flying sleigh,🦌 Reindeer
This is cut down and decorated,🎄 Christmas Tree
Santa puts these under the tree,🎁 Gifts
Best way to spend the holidays,👪 Family


Once again we see the power of zero-shot labeling. The model wasn't trained on any data specific to this example. Still amazed with how much knowledge is stored in large NLP models.

# Start an API instance

Now we'll start an API instance to run the remaining examples. The API needs a configuration file to run. The example below is simplified to only include labeling. See [this link](https://github.com/neuml/txtai#api) for a more detailed configuration example.

The API instance is started in the background.


In [ ]:
%%writefile index.yml

# Labels settings
labels:

Writing index.yml


In [ ]:
!CONFIG=index.yml nohup uvicorn "txtai.api:app" &> api.log &
!sleep 90

# JavaScript

txtai.js is available via NPM and can be installed as follows.

```bash
npm install txtai
```

For this example, we'll clone the txtai.js project to import the example build configuration.

In [ ]:
%%capture
!git clone https://github.com/neuml/txtai.js

## Create labels.js

The following file is a JavaScript version of the labels example.

In [ ]:
%%writefile txtai.js/examples/node/src/labels.js
import {Labels} from "txtai";
import {sprintf} from "sprintf-js";

const run = async () => {
    try {
        let labels = new Labels("http://localhost:8000");

        let data = ["Wears a red suit and says ho ho",
                    "Pulls a flying sleigh",
                    "This is cut down and decorated",
                    "Santa puts these under the tree",
                    "Best way to spend the holidays"];

        // List of labels
        let tags = ["🎅 Santa Clause", "🦌 Reindeer", "🍪 Cookies", "🎄 Christmas Tree", "🎁 Gifts", "👪 Family"];

        console.log(sprintf("%-40s %s", "Text", "Label"));
        console.log("-".repeat(75))

        for (let text of data) {
            let label = await labels.label(text, tags);
            label = tags[label[0].id];

            console.log(sprintf("%-40s %s", text, label));
        }
    }
    catch (e) {
        console.trace(e);
    }
};

run();


Overwriting txtai.js/examples/node/src/labels.js


## Build and run labels example





In [ ]:
%%capture
os.chdir("txtai.js/examples/node")
!npm install
!npm run build

In [ ]:
!node dist/labels.js

Text                                     Label
---------------------------------------------------------------------------
Wears a red suit and says ho ho          🎅 Santa Clause
Pulls a flying sleigh                    🦌 Reindeer
This is cut down and decorated           🎄 Christmas Tree
Santa puts these under the tree          🎁 Gifts
Best way to spend the holidays           👪 Family


The JavaScript program is showing the same results as when natively running through Python!

# Java

txtai.java integrates with standard Java build tools (Gradle, Maven, SBT). The following shows how to add txtai as a dependency to Gradle.

```gradle
implementation 'com.github.neuml:txtai.java:v4.0.0'
```

For this example, we'll clone the txtai.java project to import the example build configuration.

In [ ]:
%%capture
os.chdir("/content")
!git clone https://github.com/neuml/txtai.java

## Create LabelsDemo.java

The following file is a Java version of the labels example.

In [ ]:
%%writefile txtai.java/examples/src/main/java/LabelsDemo.java
import java.util.Arrays;
import java.util.ArrayList;
import java.util.List;

import txtai.API.IndexResult;
import txtai.Labels;

public class LabelsDemo {
    public static void main(String[] args) {
        try {
            Labels labels = new Labels("http://localhost:8000");

            List <String> data = 
                Arrays.asList("Wears a red suit and says ho ho",
                              "Pulls a flying sleigh",
                              "This is cut down and decorated",
                              "Santa puts these under the tree",
                              "Best way to spend the holidays");

            // List of labels
            List<String> tags = Arrays.asList("🎅 Santa Clause", "🦌 Reindeer", "🍪 Cookies", "🎄 Christmas Tree", "🎁 Gifts", "👪 Family");

            System.out.printf("%-40s %s%n", "Text", "Label");
            System.out.println(new String(new char[75]).replace("\0", "-"));

            for (String text: data) {
                List<IndexResult> label = labels.label(text, tags);
                System.out.printf("%-40s %s%n", text, tags.get(label.get(0).id));
            }
        }
        catch (Exception ex) {
            ex.printStackTrace();
        }
    }
}


Overwriting txtai.java/examples/src/main/java/LabelsDemo.java


## Build and run labels example

In [ ]:
os.chdir("txtai.java/examples")
!../gradlew -q --console=plain labels 2> /dev/null

Text                                     Label
---------------------------------------------------------------------------
Wears a red suit and says ho ho          🎅 Santa Clause
Pulls a flying sleigh                    🦌 Reindeer
This is cut down and decorated           🎄 Christmas Tree
Santa puts these under the tree          🎁 Gifts
Best way to spend the holidays           👪 Family


The Java program is showing the same results as when natively running through Python!

# Rust

txtai.rs is available via crates.io and can be installed by adding the following to your cargo.toml file.

```toml
[dependencies]
txtai = { version = "4.0" }
tokio = { version = "0.2", features = ["full"] }
```

For this example, we'll clone the txtai.rs project to import the example build configuration. First we need to install Rust.

In [ ]:
%%capture
os.chdir("/content")
!apt-get install rustc
!git clone https://github.com/neuml/txtai.rs

## Create labels.rs

The following file is a Rust version of the labels example.

In [ ]:
%%writefile txtai.rs/examples/demo/src/labels.rs
use std::error::Error;

use txtai::labels::Labels;

pub async fn labels() -> Result<(), Box<dyn Error>> {
    let labels = Labels::new("http://localhost:8000");

    let data = ["Wears a red suit and says ho ho",
                "Pulls a flying sleigh",
                "This is cut down and decorated",
                "Santa puts these under the tree",
                "Best way to spend the holidays"];

    println!("{:<40} {}", "Text", "Label");
    println!("{}", "-".repeat(75));

    for text in data.iter() {
        let tags = vec!["🎅 Santa Clause", "🦌 Reindeer", "🍪 Cookies", "🎄 Christmas Tree", "🎁 Gifts", "👪 Family"];
        let label = labels.label(text, &tags).await?[0].id;

        println!("{:<40} {}", text, tags[label]);
    }

    Ok(())
}

Overwriting txtai.rs/examples/demo/src/labels.rs


## Build and run labels example





In [ ]:
%%capture
os.chdir("txtai.rs/examples/demo")
!cargo build

In [ ]:
!cargo run labels

    Finished dev [unoptimized + debuginfo] target(s) in 0.07s
     Running `target/debug/demo labels`
Text                                     Label
---------------------------------------------------------------------------
Wears a red suit and says ho ho          🎅 Santa Clause
Pulls a flying sleigh                    🦌 Reindeer
This is cut down and decorated           🎄 Christmas Tree
Santa puts these under the tree          🎁 Gifts
Best way to spend the holidays           👪 Family


The Rust program is showing the same results as when natively running through Python!

# Go

txtai.go can be installed by adding the following import statement. When using modules, txtai.go will automatically be installed. Otherwise use `go get`.

```golang
import "github.com/neuml/txtai.go"
```

For this example, we'll create a standalone process for labeling. First we need to install Go.

In [ ]:
%%capture
os.chdir("/content")
!apt install golang-go
!go get "github.com/neuml/txtai.go"

## Create labels.go

The following file is a Go version of the labels example.

In [ ]:
%%writefile labels.go
package main

import (
	"fmt"
	"strings"
	"github.com/neuml/txtai.go"
)

func main() {
	labels := txtai.Labels("http://localhost:8000")

	data := []string{"Wears a red suit and says ho ho",
                   "Pulls a flying sleigh",
                   "This is cut down and decorated",
                   "Santa puts these under the tree",
                   "Best way to spend the holidays"}

	// List of labels
	tags := []string{"🎅 Santa Clause", "🦌 Reindeer", "🍪 Cookies", "🎄 Christmas Tree", "🎁 Gifts", "👪 Family"}

	fmt.Printf("%-40s %s\n", "Text", "Label")
	fmt.Println(strings.Repeat("-", 75))

	for _, text := range data {
		label := labels.Label(text, tags)
		fmt.Printf("%-40s %s\n", text, tags[label[0].Id])
	}
}

Writing labels.go


## Build and run labels example


In [ ]:
!go run labels.go

Text                                     Label
---------------------------------------------------------------------------
Wears a red suit and says ho ho          🎅 Santa Clause
Pulls a flying sleigh                    🦌 Reindeer
This is cut down and decorated           🎄 Christmas Tree
Santa puts these under the tree          🎁 Gifts
Best way to spend the holidays           👪 Family


The Go program is showing the same results as when natively running through Python!